In [27]:
import pandas as pd

matches_past = pd.read_csv("matches_past_elo.csv")
matches_future = pd.read_csv("matches_future_elo.csv")

In [28]:
# Adding a flag for if the team is playing at home or away
matches_past["is_home"] = matches_past["Venue"].astype("category").cat.codes
matches_future["is_home"] = matches_future["Venue"].astype("category").cat.codes

# Extracting hour and day of week from date and time
matches_past["Hour"] = matches_past["Time"].str.replace(r":.+", "", regex=True).fillna(0).astype(int)
matches_future["Hour"] = matches_future["Time"].str.replace(r":.+", "", regex=True).fillna(0).astype(int)

matches_past["DayOfWeek"] = pd.to_datetime(matches_past["Date"]).dt.dayofweek
matches_future["DayOfWeek"] = pd.to_datetime(matches_future["Date"]).dt.dayofweek

### Calculating the Rolling Mean of Key Stats

Calculating the rolling mean allows team form to be a factor in determining performance. I will use the form from the last 3 games for the key stats


In [29]:
def rolling_averages(group, cols, new_cols, window=3):
    group = group.sort_values("Date")
    # Compute rolling averages for the past N games (excluding current)
    rolling_stats = group[cols].rolling(window, closed="left", min_periods=1).mean()
    group[new_cols] = rolling_stats
    return group

In [30]:
cols = [
    "xG", "xGA", "Poss", "Sh_shooting", "SoT_shooting", "Dist_shooting",
    "Touches_poss", "Def Pen_poss", "Def 3rd_poss", "Mid 3rd_poss", 
    "Att 3rd_poss", "Att Pen_poss", "Succ%_poss", "PrgDist_poss", "1/3_poss",
    "SCA_gsc", "PassLive_gsc", "Tkl+Int_def", "TklW_def", "Tkl%_def"
]

new_cols = [f"avg_{c}" for c in cols]

In [31]:
# Computing a rolling average for every match in the df
matches_past = matches_past.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols, window=3))
# Dropping the team level index created by groupby, as there would be two indexes
matches_past = matches_past.droplevel("Team")

#Re-indexing our data
matches_past.index = range(matches_past.shape[0]) 

# Forward fill per team, then fill any leftover with global mean. This is to account for teams that have not played 3 games yet, or do not have data available

matches_past[new_cols] = (
    matches_past.groupby("Team")[new_cols]
    .transform(lambda g: g.ffill())
    .fillna(matches_past[new_cols].mean())
)

# Adding opponent rolling averages
opp_cols = [f"opp_{c}" for c in new_cols]

# For each match, get opponent's rolling averages at that point in time
for idx, row in matches_past.iterrows():
    opponent = row["Opponent"]
    match_date = row["Date"]
    
    # Find opponent's most recent match before this one
    opponent_recent = matches_past[
        (matches_past["Team"] == opponent) & 
        (matches_past["Date"] < match_date)
    ].sort_values("Date").tail(1)
    
    if len(opponent_recent) > 0:
        # Use opponent's rolling averages
        for new_col, opp_col in zip(new_cols, opp_cols):
            matches_past.at[idx, opp_col] = opponent_recent[new_col].iloc[0]
    else:
        # If no previous matches, use global mean
        for new_col, opp_col in zip(new_cols, opp_cols):
            matches_past.at[idx, opp_col] = matches_past[new_col].mean()

matches_past

/var/folders/x7/sl4zn7qx64s6wq122_6jw9_00000gn/T/ipykernel_80180/2837806767.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_past = matches_past.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols, window=3))


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,opp_avg_Att 3rd_poss,opp_avg_Att Pen_poss,opp_avg_Succ%_poss,opp_avg_PrgDist_poss,opp_avg_1/3_poss,opp_avg_SCA_gsc,opp_avg_PassLive_gsc,opp_avg_Tkl+Int_def,opp_avg_TklW_def,opp_avg_Tkl%_def
0,2023-08-12,20:00,Eredivisie,Matchweek 1,Sat,Home,W,4,1,Heracles Almelo,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302
1,2023-08-19,16:30,Eredivisie,Matchweek 2,Sat,Away,D,2,2,Excelsior,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302
2,2023-08-24,21:00,Europa Lg,Play-off round,Thu,Away,W,4,1,Ludogorets,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302
3,2023-08-31,20:00,Europa Lg,Play-off round,Thu,Home,L,0,1,Ludogorets,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302
4,2023-09-03,14:30,Eredivisie,Matchweek 4,Sun,Away,D,0,0,Fortuna Sittard,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,2025-05-18,19:00,La Liga,Matchweek 37,Sun,Away,W,3,2,Barcelona,...,291.333333,39.333333,48.333333,1493.000000,28.00000,42.333333,33.666667,23.000000,6.666667,55.100000
2786,2025-05-25,16:15,La Liga,Matchweek 38,Sun,Home,W,4,2,Sevilla,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302
2787,2025-08-15,21:30,La Liga,Matchweek 1,Fri,Home,W,2.0,0.0,Oviedo,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302
2788,2025-08-24,19:30,La Liga,Matchweek 2,Sun,Home,W,5.0,0.0,Girona,...,175.683238,27.901740,44.678598,1082.985077,15.16662,26.691413,19.978226,20.949043,10.089485,51.678302


### Implementing the most recent rolling stats into the matches_future dataset

In [32]:
# Adding the most recent rolling stats to the matches_future dataset
latest_stats = (
    matches_past.sort_values("Date")
    .groupby("Team")
    .tail(1)[["Team"] + new_cols]
    .set_index("Team")
)

matches_future = matches_future.merge(
    latest_stats, left_on="Team", right_index=True, how="left"
)

# Adding opponent rolling averages to future matches
matches_future = matches_future.merge(
    latest_stats.add_prefix("opp_"), left_on="Opponent", right_index=True, how="left"
)

matches_future[matches_future["Team"] == "Real Madrid"]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,opp_avg_Att 3rd_poss,opp_avg_Att Pen_poss,opp_avg_Succ%_poss,opp_avg_PrgDist_poss,opp_avg_1/3_poss,opp_avg_SCA_gsc,opp_avg_PassLive_gsc,opp_avg_Tkl+Int_def,opp_avg_TklW_def,opp_avg_Tkl%_def
14,2025-09-13,16:15,La Liga,Matchweek 4,Sat,Away,NaN,NaN,NaN,Real Sociedad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,2025-09-16,21:00,Champions Lg,League phase,Tue,Home,NaN,NaN,NaN,Marseille,...,193.333333,28.333333,37.300000,1101.000000,12.666667,31.666667,23.666667,21.000000,11.000000,58.333333
76,2025-09-20,16:15,La Liga,Matchweek 5,Sat,Home,NaN,NaN,NaN,Espanyol,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,2025-09-23,21:30,La Liga,Matchweek 6,Tue,Away,NaN,NaN,NaN,Levante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,2025-09-27,16:15,La Liga,Matchweek 7,Sat,Away,NaN,NaN,NaN,Atlético Madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,2025-09-30,21:45,Champions Lg,League phase,Tue,Away,NaN,NaN,NaN,Qaırat Almaty,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,2025-10-04,21:00,La Liga,Matchweek 8,Sat,Home,NaN,NaN,NaN,Villarreal,...,219.000000,38.000000,46.300000,1303.333333,19.333333,34.000000,26.666667,21.666667,7.333333,48.733333
233,2025-10-19,21:00,La Liga,Matchweek 9,Sun,Away,NaN,NaN,NaN,Getafe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,2025-10-22,21:00,Champions Lg,League phase,Wed,Home,NaN,NaN,NaN,Juventus,...,212.333333,37.333333,39.766667,1145.666667,20.000000,40.333333,31.000000,27.666667,10.666667,53.066667
294,2025-10-26,NaN,La Liga,Matchweek 10,Sun,Home,NaN,NaN,NaN,Barcelona,...,249.333333,38.333333,49.433333,1410.333333,16.666667,38.666667,29.666667,19.666667,7.333333,52.733333


### Writing the final dataset as a csv so it can be used to predict the model

In [33]:
matches_past.to_csv("matches_past_features.csv", index=False)
matches_future.to_csv("matches_future_features.csv", index=False)